In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import sys
import shutil
import splitfolders
from PIL import Image

In [3]:
''''''
try:
    shutil.rmtree('./datain/test_big/')
except OSError as e:
    print("Error: %s - %s." % (e.filename, e.strerror))


In [4]:

try:
    shutil.rmtree('./datain/test_small/')
except OSError as e:
    print("Error: %s - %s." % (e.filename, e.strerror))

In [5]:
# train, test split
splitfolders.ratio('./datain/', output="./splitted_data", ratio=(0.7, 0.3))

In [6]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('./splitted_data/train/',
                                                 target_size = (64,64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 9612 images belonging to 2 classes.


In [7]:
training_set.class_indices

{'train-smoke': 0, 'train_fire': 1}

In [8]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('./splitted_data/val/',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 4121 images belonging to 2 classes.


In [9]:
# cnn
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=7, activation='relu', input_shape=[64,64,3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=256, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [10]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 58, 58, 32)        4736      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 29, 29, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 5408)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               1384704   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2

In [11]:
cnn.fit(x = training_set, epochs = 7 ,validation_data=test_set)

Epoch 1/7
301/301 [==============================] - 429s 1s/step - loss: 0.0829 - accuracy: 0.9742 - val_loss: 0.0253 - val_accuracy: 0.9925
Epoch 2/7
301/301 [==============================] - 315s 1s/step - loss: 0.0553 - accuracy: 0.9849 - val_loss: 0.0333 - val_accuracy: 0.9886
Epoch 3/7
301/301 [==============================] - 318s 1s/step - loss: 0.0506 - accuracy: 0.9866 - val_loss: 0.0233 - val_accuracy: 0.9927
Epoch 4/7
301/301 [==============================] - 316s 1s/step - loss: 0.0327 - accuracy: 0.9908 - val_loss: 0.0104 - val_accuracy: 0.9973
Epoch 5/7
301/301 [==============================] - 318s 1s/step - loss: 0.0398 - accuracy: 0.9908 - val_loss: 0.0481 - val_accuracy: 0.9823
Epoch 6/7
301/301 [==============================] - 314s 1s/step - loss: 0.0375 - accuracy: 0.9904 - val_loss: 0.0078 - val_accuracy: 0.9978
Epoch 7/7
301/301 [==============================] - 325s 1s/step - loss: 0.0260 - accuracy: 0.9933 - val_loss: 0.0178 - val_accuracy: 0.9961
